In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

from tqdm import tqdm
from sklearn import datasets
import random

In [3]:
import mylibrary.datasets as datasets

In [4]:
device = torch.device("cuda:0")
# device = torch.device("cuda:1")
# device = torch.device("cpu")

In [5]:
mnist = datasets.FashionMNIST()
# mnist.download_mnist()
# mnist.save_mnist()
train_data, train_label_, test_data, test_label_ = mnist.load()

train_data = train_data / 255.
test_data = test_data / 255.

train_size = len(train_label_)

In [6]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)
test_label = torch.LongTensor(test_label_)

In [7]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)

In [8]:
input_size = 784
output_size = 10

In [9]:
class MNIST_Dataset(data.Dataset):
    
    def __init__(self, data, label):
        self.data = data
        self.label = label
        
#         self.label = mask.type(torch.float32).reshape(-1,1)
        self._shuffle_data_()
        
    def __len__(self):
        return len(self.data)
    
    def _shuffle_data_(self):
        randidx = random.sample(range(len(self.data)), k=len(self.data))
        self.data = self.data[randidx]
        self.label = self.label[randidx]
    
    def __getitem__(self, idx):
        img, lbl = self.data[idx], self.label[idx]
        return img, lbl

In [10]:
train_dataset = MNIST_Dataset(train_data, train_label)
test_dataset = MNIST_Dataset(test_data, test_label)

In [11]:
learning_rate = 0.0003
batch_size = 50

In [12]:
train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

## Making of matrix factorized layer

In [13]:
class PairBilinear(nn.Module):
    def __init__(self, dim, grid_width):
        super().__init__()
        self.dim = dim
        self.grid_width = grid_width
        
        self.num_pairs = self.dim // 2
        along_row = torch.linspace(0, 1, self.grid_width).reshape(1, -1)
        along_col = torch.linspace(0, 1, self.grid_width).reshape(-1, 1)
        self.Y = torch.stack([along_row+along_col*0, along_row*0+along_col])
        self.Y = torch.repeat_interleave(self.Y.unsqueeze(0), self.num_pairs, dim=0)
        self.Y = nn.Parameter(self.Y)
        
        self.pairW = torch.eye(2).unsqueeze(0).repeat_interleave(self.num_pairs, dim=0)*0.5
        self.pairW = nn.Parameter(self.pairW)
    
    def forward(self, x):
        bs = x.shape[0]
        x = x.view(-1, 2)
        
        _wi = torch.arange(self.num_pairs).repeat(bs)
        _W = self.pairW[_wi]
        x = torch.bmm(x.unsqueeze(1), _W).squeeze(1)
        
        x = x*self.grid_width
        index = torch.clamp(x.data, 0, self.grid_width-2)
        index = torch.floor(index)
        x = x-index
        
        index = (index.repeat_interleave(2, dim=0))
        
        _bi = torch.arange(bs).repeat_interleave(self.num_pairs*2)
        _gi = torch.arange(self.num_pairs).repeat_interleave(2).repeat(bs)
        _pi = torch.LongTensor([0,1]).repeat(bs*self.num_pairs)
        _xc, _yc = tuple(index.type(torch.long).t())

        f00 = self.Y[_gi, _pi, _xc, _yc]
        f01 = self.Y[_gi, _pi, _xc, _yc+1]
        f10 = self.Y[_gi, _pi, _xc+1, _yc]
        f11 = self.Y[_gi, _pi, _xc+1, _yc+1]
        
        a00 = f00
        a10 = f10-f00
        a01 = f01-f00
        a11 = f11-f10-f01+f00
        
        ##### this doubles the multiplication for x,y
#         x = x.repeat_interleave(2, dim=0)
#         y = a00 + x[:, 0]*a10 + x[:, 1]*a01 + x[:, 0]*x[:, 1]*a11

        ##### this repeats in individual way
        _x, _y = x[:, 0].repeat_interleave(2), x[:, 1].repeat_interleave(2)
        y = a00 + _x*a10 + _y*a01 + _x*_y*a11

        ### now 
        y = y.view(bs, -1)
        return y

In [14]:
class BiasLayer(nn.Module):
    def __init__(self, dim, init_val=0):
        super().__init__()
        self.bias = nn.Parameter(torch.ones(dim)*init_val)
        
    def forward(self, x):
        return x+self.bias

In [15]:
class FactorizedPairBilinearSpline(nn.Module):
    
    def __init__(self, input_dim, grid_width):
        super().__init__()
        assert input_dim%2 == 0, "Input dim must be even number"
        self.input_dim = input_dim
        num_layers = int(np.ceil(np.log2(8)))
            
        self.facto_nets = []
        self.idx_revidx = []
        for i in range(num_layers):
            idrid = self.get_pair(self.input_dim, i+1)
            net = PairBilinear(self.input_dim, grid_width)
            self.facto_nets.append(net)
            self.idx_revidx.append(idrid)
        self.facto_nets = nn.ModuleList(self.facto_nets)
            
    def get_pair(self, inp_dim, step=1):
        dim = 2**int(np.ceil(np.log2(inp_dim)))
        assert isinstance(step, int), "Step must be integer"

        blocks = (2**step)
        range_ = dim//blocks
        adder_ = torch.arange(0, range_)*blocks

        pairs_ = torch.Tensor([0, blocks//2])
        repeat_ = torch.arange(0, blocks//2).reshape(-1,1)
        block_map = (pairs_+repeat_).reshape(-1)

        reorder_for_pair = (block_map+adder_.reshape(-1,1)).reshape(-1)
        indx = reorder_for_pair.type(torch.long)
        indx = indx[indx<inp_dim]

        rev_indx = torch.argsort(indx)
        return indx, rev_indx
    
    def forward(self, x):
        ## swap first and then forward and reverse-swap
        y = x
        for i in range(len(self.facto_nets)):
            idx, revidx = self.idx_revidx[i]
            y = y[:, idx]
            y = self.facto_nets[i](y) 
            y = y[:, revidx]
        y = x + y ## this is residual addition... remove if only want feed forward
        return y

In [16]:
pfL = FactorizedPairBilinearSpline(100, 5)

In [17]:
pfL(torch.randn(2, 100))

tensor([[ 1.1910, -0.4813,  0.7383,  0.0850,  2.2843,  2.3168, -2.2500, -0.0840,
          0.8291, -0.9372, -1.3600,  0.8095, -0.2464, -0.7357, -0.6284,  0.4242,
         -1.8535,  1.0069,  0.7202, -0.4203, -1.5171,  0.1981,  0.1598, -0.8402,
         -1.2538,  0.7275,  0.9275,  1.2101,  0.5872,  0.1295,  1.0091,  0.5465,
          0.1957,  1.0832, -0.2674,  0.5134,  0.4542, -0.2545,  2.5247, -0.1317,
         -1.1916, -0.2517, -1.7568, -1.3354, -1.5816,  0.5078,  0.3484, -1.0522,
         -0.6264, -1.3891, -0.8363, -1.1435, -0.6195, -0.9505,  0.4569, -0.4654,
          0.5972,  0.4968, -0.5019,  0.4638,  0.0665, -0.5245, -0.0783, -0.1177,
         -1.0549, -0.8273,  0.0999, -0.8043, -0.1415, -0.1666, -0.7239, -0.3350,
         -0.3849, -0.0523, -0.6248, -1.5778, -0.8935, -0.3656,  0.0641, -1.0394,
         -2.0750, -1.3378,  0.0964,  1.8636, -0.6408, -1.1121, -1.1916,  0.2749,
          1.0441,  1.0171,  0.5025,  0.8921,  1.0660, -0.2129,  1.9718, -0.8817,
          0.1989, -0.1899,  

In [18]:
param_count = sum([torch.numel(p) for p in pfL.parameters()])
param_count

8100

In [44]:
class FactorNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.bias = BiasLayer(784)
        self.la1 = FactorizedPairBilinearSpline(784, grid_width=10)
        self.bn1 = nn.BatchNorm1d(784)
        self.fc = nn.Linear(784, 10)
        
    def forward(self, x):
        x = self.bias(x)
        x = self.bn1(self.la1(x))
        x = self.fc(x)
        return x

In [45]:
# class OrdinaryNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.la1 = nn.Linear(784, 1024, bias=False)
#         self.bn1 = nn.BatchNorm1d(1024)
#         self.la2 = nn.Linear(1024, 1024)
#         self.bn2 = nn.BatchNorm1d(1024, bias=False)
#         self.la3 = nn.Linear(1024, 10)
        
#     def forward(self, x):
#         x = self.bn1(self.la1(x))
#         x = torch.relu(x)
#         x = self.bn2(self.la2(x))
#         x = torch.relu(x)
#         x = self.la3(x)
#         return x

In [48]:
model = FactorNet()
param_count = sum([torch.numel(p) for p in model.parameters()])
param_count

250106

In [49]:
# model = OrdinaryNet()
# param_count1 = sum([torch.numel(p) for p in model.parameters()])
# param_count1, param_count1/param_count

### Model Development

In [50]:
torch.manual_seed(0)
model = FactorNet().to(device)
# model = OrdinaryNet().to(device)
model

FactorNet(
  (bias): BiasLayer()
  (la1): FactorizedPairBilinearSpline(
    (facto_nets): ModuleList(
      (0): PairBilinear()
      (1): PairBilinear()
      (2): PairBilinear()
    )
  )
  (bn1): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Linear(in_features=784, out_features=10, bias=True)
)

In [51]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [52]:
print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  250106


In [53]:
losses = []
train_accs = []
test_accs = []
EPOCHS = 20

for epoch in range(EPOCHS):
    
    train_acc = 0
    train_count = 0
    for xx, yy in tqdm(train_loader):
        xx, yy = xx.to(device), yy.to(device)

        yout = model(xx)
        loss = criterion(yout, yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(float(loss))

        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        train_acc += correct
        train_count += len(outputs)

    train_accs.append(float(train_acc)/train_count*100)
    train_acc = 0
    train_count = 0

    print(f'Epoch: {epoch},  Loss:{float(loss)}')
    test_count = 0
    test_acc = 0
    for xx, yy in tqdm(test_loader):
        xx, yy = xx.to(device), yy.to(device)
        with torch.no_grad():
            yout = model(xx)
        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        test_acc += correct
        test_count += len(xx)
    test_accs.append(float(test_acc)/test_count*100)
    print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
    print()

### after each class index is finished training
print(f'\t-> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}')

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 0,  Loss:0.5092830061912537


  0%|          | 0/1200 [00:00<?, ?it/s]

Train Acc:75.91%, Test Acc:81.95%



  0%|          | 0/200 [00:00<?, ?it/s]

...

Train Acc:92.03%, Test Acc:88.28%



  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 19,  Loss:0.1551836133003235


100%|██████████| 200/200 [00:02<00:00, 93.84it/s] 

Train Acc:92.17%, Test Acc:88.29%

	-> Train Acc 92.17166666666667 ; Test Acc 88.36


In [ ]:
## stats: 20 epochs
### for factor-net: 5grid : 73706-> 100%|██████████| 1200/1200 [00:24<00:00, 48.44it/s]
########### -> Train Acc 90.3367 ; Test Acc 88.06

### for factor-net: 50grid : 5894906-> 100%|██████████| 1200/1200 [00:28<00:00, 42.74it/s]
########### -> Train Acc 99.985 ; Test Acc 85.85

### for factor-net: 10grid : 250106-> 100%|██████████| 1200/1200 [00:24<00:00, 48.11it/s]
########### -> Train Acc 92.17167 ; Test Acc 88.36